بسم الله الرحمن الرحيم

**Deuxième extraction**

Détails 2/2

Dans ce noteboook, on essaye d'ajouter la date de décès de l'auteur, une description du livre et des détails sur l'édition, et le domaine du livre.


On pourra joindre sur le lien internet avec le fichier crée dans la première extraction.

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup 
import numpy as np

### Etape 1 : Aller chercher les urls des livres

In [3]:
##Aller chercher les urls des livres
urls = []
new_urls = []
to_supp =[]
NB_MAX_PAGES = 15
for i in range(1,NB_MAX_PAGES):
    i = str(i)
    r = requests.get('https://www.sifatusafwa.com/fr/fiqh-maliki/?page=' + i)
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find('section', id = 'products')
    #s = s.find_all('div', class_ = 'products product_list row grid')
    #lines = s.find('h3', class_ = 's_title_block flex_child')
    #Affichage du contenu de la page
    #print(lines)

    #Extraire les liens
    
    for line in s.findAll('a'):
        urls.append(line.get('href'))#line.text.split(\'n') est une liste de chaine de caractères, donc [0] est 
        #seulement le str.
        #print(line.text.split('\n'))#line.text enlève les balises et les choses propres à html/css. Le .split split sur les \n
    #Tous les liens sont en double/ triple... on va les faire apparaitre qu'une seule fois.
    urls = list(np.unique(urls))
    #urls
    # On va supprimer les liens qui ne renvoient pas vers des livres : 
    #  - Ceux qui finissent pas '.asc' ou 'desc
    #  - Les liens de page ("?page=2", "?page=3"...) et le lien 'fiqh-maliki/'
    #  - les "javascript"
    #Suppression des liens ne renvoyant pas vers des livres.

for link in urls:
    if (link.endswith('asc'))==False and \
        (link.endswith('.desc'))==False and \
        (link.startswith('https://www.sifatusafwa.com/fr/fiqh-maliki/?page=')) == False and \
            (link.startswith('https://www.sifatusafwa.com/fr/fiqh-maliki/?rewrite'))== False:
        new_urls.append(link)            
for i in range(3):
    try:
        new_urls.remove('javascript:')
        new_urls.remove('javascript:;')
        new_urls.remove('https://www.sifatusafwa.com/fr/fiqh-maliki/')
    except ValueError:
        continue
new_urls = pd.DataFrame(new_urls)
del urls
new_urls = new_urls.values.T.tolist()[0]
#on enlève tout ce qu'on a rajouté maintenant qu'on a pu faire la comparaison avec ce qu'il y avait
#a supprimer
urls = list(np.unique(new_urls))

Implémentation du prix & de l'auteur : on essaye pour le premier lien
On essaye d'utiliser la balise `<'a', class = 'pro_extra_info_brand'>`pour les auteurs et `span`, `class = 'price` pour les prix

In [4]:
url = urls[0]
prices = []
auteurs = []
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', id ='main')

#On vérifie si l'auteur du livre est disponible:
# url = 'https://www.sifatusafwa.com/fr/fiqh-maliki/ikhtiyarat-al-hafidh-ibn-abdil-barr-al-fiqhiya.html'
if s.find("a", class_= 'pro_extra_info_brand') != None:
    s = s.text.split('\n')
    auteurs.append(s)
#On met le prix    
price = s.find("span", class_ = 'price').text
prices.append(price)

On peut essayer d'ajouter la date de mort de l'auteur رحمه الله

In [5]:
#Affichaage des résulatats:
url = "https://www.sifatusafwa.com/fr/livres-de-salaf-sur-la-sounnah/charh-as-sounnah-de-l-imam-al-barbahary-100-harakat.html#/22-edition-saoudienne"
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', id ='main')
s_auteur = s.find("a", class_= 'pro_extra_info_brand').text
s_nomauteur = s_auteur.split('\n')[1].split(' ')[0] #C'est juste le nom
s_dateauteur = s_auteur.split('(')[1].split('H)')[0] #C'est juste la date 
print("date:", s_dateauteur, "nom:",s_nomauteur)

date: 329 nom: Al-Barbahaary


Des fois, le nom de l'auteur n'est pas mentionné, on essaye:

In [6]:
#Affichaage des résulatats:
url = "https://www.sifatusafwa.com/fr/grammaire-nahou/charh-al-mouqaddimah-al-ajroumiyyah-ajouramiyyah-salih-al-osaimi.html"
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', id ='main')
s_auteur = s.find("a", class_= 'pro_extra_info_brand').text
s_nomauteur = s_auteur.split('\n')[1].split(' ')[0] #C'est juste le nom
try:
    s_dateauteur = s_auteur.split('(')[1].split('H)')[0] #C'est juste la date 
    print("date:", s_dateauteur, "nom:", s_nomauteur)
except IndexError:
    pass

## Etape 2: Boucle sur les urls

In [30]:
dfs = []
prices = []
auteurs = []
dates = []
domaines = []
i = 0
for url in urls:
    i = i+1
    print('url numero', i, '/', len(urls), (i/len(urls))*100, "%")
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find('section', id ='main')

    #On vérifie si l'auteur du livre est disponible:
    s_auteur = s.find("a", class_= 'pro_extra_info_brand') #Si on divise pas en s_auteur et s_prix le find deviendra une liste car ce sera un find d'un find...
    
    #Soit pas d'auteur, soit auteur avec date, soit auteur sans date, soit auteur avec date mais deux
    #fois la parenthèse.

    if  s_auteur != None:#Auteur existe
        s_auteur = s_auteur.text         
        #Le try c'est juste pour couvrir tous les cas de date de décès possible
        try:#Cas 1 : La date existe : on l'ajoute
            s_dateauteur = s_auteur.split('(')[1].split('H)')[0]#s_auteur.split('(') = ['al qadhi iyyadh ', '544H)']
            dates.append(int(s_dateauteur))            

           #Ibn Rushd (Al-Jadd) (520H)	: split('\n') = ['', 'ibn rushd (Al hafid)', '']
        except IndexError:#Cas 2: Pas de date de décès disponible
            dates.append('') #On ne met pas de date pour pas qu'il y ait de décalage: un auteur avec la date de décès de l'auteur 
            #suivant
        except ValueError:#Cas 3: deux fois la parenthèse, il ne peut pas convertir en entier: c'est une 
            #chaine de caractères
            dates.append('') #Avantageux de remplir les auteurs après avoir vérifié la date car ibn rushd (al hafid) est coupé

        auteur = (s_auteur.split('\n')[1].split('(')[0])#s_auteur.split('\n) = ['', 'ibn rushd (Al hafid)', '']
         # ou ['', 'al qadi iyadd','(544)]
        auteurs.append(auteur) #Pour pas qu'on ait un espace à la fin.
            
    else: #Pas d'auteur
        auteurs.append('Auteur non disponible')
        dates.append('') #pour pas qu'il y ait de décalage: un auteur avec la date de décès de l'auteur suivant
    
    #On met le prix    
    s_prix = s.find("span", class_ = 'price')
    try:
        price = s_prix.text
        prices.append(price)

    except AttributeError: #s.find eset une liste et .text n'existe pas pour une liste: normalement plus d'erreur
        print(url, type(s))
    #Ajout du domaine:
    domaine = url.split('/')[4]
    domaines.append(domaine)

url numero 1 / 136 0.7352941176470588 %
url numero 2 / 136 1.4705882352941175 %
url numero 3 / 136 2.2058823529411766 %
url numero 4 / 136 2.941176470588235 %
url numero 5 / 136 3.6764705882352944 %
url numero 6 / 136 4.411764705882353 %
url numero 7 / 136 5.147058823529411 %
url numero 8 / 136 5.88235294117647 %
url numero 9 / 136 6.61764705882353 %
url numero 10 / 136 7.352941176470589 %
url numero 11 / 136 8.088235294117647 %
url numero 12 / 136 8.823529411764707 %
url numero 13 / 136 9.558823529411764 %
url numero 14 / 136 10.294117647058822 %
url numero 15 / 136 11.029411764705882 %
url numero 16 / 136 11.76470588235294 %
url numero 17 / 136 12.5 %
url numero 18 / 136 13.23529411764706 %
url numero 19 / 136 13.970588235294118 %
url numero 20 / 136 14.705882352941178 %
url numero 21 / 136 15.441176470588236 %
url numero 22 / 136 16.176470588235293 %
url numero 23 / 136 16.911764705882355 %
url numero 24 / 136 17.647058823529413 %
url numero 25 / 136 18.38235294117647 %
url numero 2

In [8]:
url = "https://www.sifatusafwa.com/en/generalist-fiqh/bidaayyah-al-mujtahid-wa-nihaayyah-al-muqtasid-ibn-rushd.html"
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', id ='main')
s_auteur = s.find("a", class_= 'pro_extra_info_brand').text
s_auteur.split('\n')[1].split('(')[0]


'Ibn Rushd '

In [31]:
pd.concat((pd.DataFrame(prices), pd.DataFrame(auteurs), pd.DataFrame(dates), pd.DataFrame(domaines),\
    pd.DataFrame(urls)), axis = 1).sample(25)

,0,0,0,0,0
132,"28,50 €",Imam Malik,179,recueils-classiques-de-hadith,https://www.sifatusafwa.com/fr/recueils-classi...
78,"22,00 €",Auteur non disponible,,fiqh-maliki,https://www.sifatusafwa.com/fr/fiqh-maliki/int...
122,"62,90 €",Al-Qarafi,684,oussoul-al-fiqh,https://www.sifatusafwa.com/fr/oussoul-al-fiqh...
134,"12,00 €",Auteur non disponible,,sectes-et-innovations,https://www.sifatusafwa.com/fr/sectes-et-innov...
64,"55,00 €",Auteur non disponible,,fiqh-maliki,https://www.sifatusafwa.com/fr/fiqh-maliki/cha...
0,"16,90 €",Auteur non disponible,,al-ilm-wa-talab-al-ilm,https://www.sifatusafwa.com/fr/al-ilm-wa-talab...
12,"75,00 €",Al-Qadi 'Iyad,544,explications-de-recueils,https://www.sifatusafwa.com/fr/explications-de...
49,"229,00 €",Ibn 'Arafah,803,fiqh-maliki,https://www.sifatusafwa.com/fr/fiqh-maliki/al-...
106,"27,90 €",Khalil Ibn Ishaq,776,fiqh-maliki,https://www.sifatusafwa.com/fr/fiqh-maliki/mou...
99,"5,50 €",Ibn Abi Zayd al-Qayrawani,386,fiqh-maliki,https://www.sifatusafwa.com/fr/fiqh-maliki/mat...


## Etape 3: Implémentation de la description du livre

On va utiliser la balise `class = 'product-information'` qui contient le résumé d'un livre, exemple:

In [32]:
url = urls[0]
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', id = 'main')
print(s.find(class_ = 'product-information').text.split('\n'))



['', 'Livre très utile et important pour l’Etudiant\xa0: Les définitions des termes juridiques au sein des différents Madhhab, ainsi que les abréviations ou surnom utilisés par chacun des Madhahib (exemple\xa0: cheikh al islam peut renvoyer à des imams différents selon les madhahib).', 'Une excellente recherche que nous vous recommandons, car connaitre les termes spécifiques est une clé pour pouvoir lire les livres et les comprendre.', '', '', '', '', '', '', ' ', '', '', '', '', '', '', '16,90\xa0€', 'TTC', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Derniers articles en stock', '', '', '', '', '', '', '', '', 'Ajouter au panier', '', '', '', '', '', '', '', '', '', 'Ajouter à la liste de souhaits', '', '', '', '', '', '', '', 'Référence: ', '18-HZM-052', '', '', 'Fiche technique', '', 'Titre', 'مصطلحات المذاهب الفقهية وأسرار الفقه المرموز في الأعلام والكتب والآراء والترجيحات', '', '', 'Auteur', 'مريم محمد صالح الظفيري', '', '', 'Pages', '457', '', '', 'Édition', 'دار ابن

On va juste prendre les chaines de caractères 1, et toutes jusqu'à rencontrer un `""`.

In [33]:
char = 1
description = ""
s_description = s.find(class_ = 'product-information').text.split('\n')
while s_description[char] != "":
    description = description + s_description[char] + '\n'
    char = char + 1
description

'Livre très utile et important pour l’Etudiant\xa0: Les définitions des termes juridiques au sein des différents Madhhab, ainsi que les abréviations ou surnom utilisés par chacun des Madhahib (exemple\xa0: cheikh al islam peut renvoyer à des imams différents selon les madhahib).\nUne excellente recherche que nous vous recommandons, car connaitre les termes spécifiques est une clé pour pouvoir lire les livres et les comprendre.\n'

On peut ajouter la description spécifique à l'édition: on utilise la classe `class = 'rte'`

In [34]:
url = 'https://www.sifatusafwa.com/fr/al-ilm-wa-talab-al-ilm/iqtidaa-al-ilm-al-amal-de-l-imam-al-khatib-al-baghdadi.html#/27-edition-couverture_cuir'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', id = 'main')
s_editiondescription =  s.find(class_ = 'rte')
if s_editiondescription!= None:
    print(s_editiondescription.text.split('\n')[0])
else:
    print('none', url)

Très belle édition, couverture rigide, grand format (17x24cm), vérifié à partir de 2 manuscrits, Harakat en partie.


On peut faire tourner la boucle sur toutes les pages, 

In [41]:
dfs = []
descriptions = []
descriptions_edition = []
i = 0
for url in urls:
    i = i+1
    if i%10 == 0:
        print('url numero', i, '/', len(urls), (i/len(urls))*100, "%")
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find('section', id ='main')

    #On atteint le stade où les guillemets ne sont plus disponibles:
    # url = 'https://www.sifatusafwa.com/fr/fiqh-maliki/ikhtiyarat-al-hafidh-ibn-abdil-barr-al-fiqhiya.html'
    char = 1
    description = ""
    s_description = s.find(class_ = 'product-information')
    try:
        s_description = s_description.text.split('\n')
        while s_description[char] != "":
            description = description + s_description[char] + '\n'
            char = char + 1
            descriptions.append(description)
    except AttributeError as e:
        print('erreur url', url, e)#ces liens ne renvoient plus vers des livres, pas important

        
    #description d'édition: table avec juste ce qu'il faut car plusieurs editions pas toujours disponible, on fera une jointure sur le site si 
    # besoin. On met donc le site dans le tableau.
    
    s_editiondescription =  s.find(class_ = 'rte')
    if s_editiondescription!= None:
        descriptions_edition.append([s_editiondescription.text.split('\n')[0], url])

url numero 10 / 136 7.352941176470589 %
url numero 20 / 136 14.705882352941178 %
url numero 30 / 136 22.058823529411764 %
url numero 40 / 136 29.411764705882355 %
url numero 50 / 136 36.76470588235294 %
url numero 60 / 136 44.11764705882353 %
url numero 70 / 136 51.470588235294116 %
url numero 80 / 136 58.82352941176471 %
url numero 90 / 136 66.17647058823529 %
url numero 100 / 136 73.52941176470588 %
url numero 110 / 136 80.88235294117648 %
erreur url https://www.sifatusafwa.com/fr/fiqh-maliki/َqa-idah-fi-tafdil-madhhab-imam-ahl-al-madina-ibn-taymiyya.html#/143-edition-syrienne_qualité_saoudienne 'NoneType' object has no attribute 'text'
erreur url https://www.sifatusafwa.com/fr/fiqh-maliki/َqa-idah-fi-tafdil-madhhab-imam-ahl-al-madina-ibn-taymiyya.html#/21-edition-egyptienne 'NoneType' object has no attribute 'text'
url numero 120 / 136 88.23529411764706 %
url numero 130 / 136 95.58823529411765 %


In [36]:
M_supplement = pd.concat((pd.DataFrame(prices),\
            pd.DataFrame(auteurs),\
            pd.DataFrame(dates),\
            pd.DataFrame(descriptions),\
                pd.DataFrame(domaines),\
            pd.DataFrame(urls)), axis = 1)
M_supplement.columns = ['prix', 'auteur', 'dates','domaines', 'description', 'lien']
M_supplement.head()

,prix,auteur,dates,domaines,description,lien
0,"16,90 €",Auteur non disponible,,Livre très utile et important pour l’Etudiant ...,al-ilm-wa-talab-al-ilm,https://www.sifatusafwa.com/fr/al-ilm-wa-talab...
1,"32,90 €",Auteur non disponible,,Livre très utile et important pour l’Etudiant ...,aqida-croyance,https://www.sifatusafwa.com/fr/aqida-croyance/...
2,"21,00 €",Auteur non disponible,,Excellente recherche universitaire qui rapport...,aqida-croyance,https://www.sifatusafwa.com/fr/aqida-croyance/...
3,"4,50 €",Auteur non disponible,,Excellente recherche universitaire qui rapport...,biographies,https://www.sifatusafwa.com/fr/biographies/mal...
4,"15,00 €",Imam Malik,179,Excellente recherche universitaire qui rapport...,biographies,https://www.sifatusafwa.com/fr/biographies/mal...


In [51]:
M_supplement['auteur'].str.split('(')

0      [Auteur non disponible]
1      [Auteur non disponible]
2      [Auteur non disponible]
3      [Auteur non disponible]
4                [Imam Malik ]
                ...           
331                        NaN
332                        NaN
333                        NaN
334                        NaN
335                        NaN
Name: auteur, Length: 336, dtype: object

## Etape 4 : On peut réunir toutes ces données et renommer les colonnes

In [37]:
M_supplement = pd.concat((pd.DataFrame(prices),\
            pd.DataFrame(auteurs),\
            pd.DataFrame(dates),\
            pd.DataFrame(domaines),\
            pd.DataFrame(descriptions),\
            pd.DataFrame(urls)), axis = 1)
M_supplement.head()

,0,0,0,0,0,0
0,"16,90 €",Auteur non disponible,,al-ilm-wa-talab-al-ilm,Livre très utile et important pour l’Etudiant ...,https://www.sifatusafwa.com/fr/al-ilm-wa-talab...
1,"32,90 €",Auteur non disponible,,aqida-croyance,Livre très utile et important pour l’Etudiant ...,https://www.sifatusafwa.com/fr/aqida-croyance/...
2,"21,00 €",Auteur non disponible,,aqida-croyance,Excellente recherche universitaire qui rapport...,https://www.sifatusafwa.com/fr/aqida-croyance/...
3,"4,50 €",Auteur non disponible,,biographies,Excellente recherche universitaire qui rapport...,https://www.sifatusafwa.com/fr/biographies/mal...
4,"15,00 €",Imam Malik,179,biographies,Excellente recherche universitaire qui rapport...,https://www.sifatusafwa.com/fr/biographies/mal...


In [38]:
#Renommer les colonnes:
M_supplement.columns = ['prix', 'auteur', 'dates', 'domaines', 'description', 'lien']

In [39]:
M_edition = pd.DataFrame(descriptions_edition)
M_edition.columns = ['description_edition', 'lien']
M_edition.head()

,description_edition,lien
0,,https://www.sifatusafwa.com/fr/explications-de...
1,,https://www.sifatusafwa.com/fr/explications-de...
2,Le charh d'un des plus grands savants de la Su...,https://www.sifatusafwa.com/fr/explications-de...
3,Le charh d'un des plus grands savants de la Su...,https://www.sifatusafwa.com/fr/explications-de...
4,Cette explication en 2 tomes est à la base une...,https://www.sifatusafwa.com/fr/explications-de...


In [40]:
M_supplement.to_csv('fiqh-maliki-supplement.csv')
M_edition.to_csv('fiqh-maliki_edition.csv')

## Fin و الحمدلله